In [14]:
import imutils
import time
import cv2
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch

In [15]:
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.utils.misc import Timer

In [3]:
img_1 = cv2.imread("image/1.jpg")
img_2 = cv2.imread("image/2.jpg")
img_3 = cv2.imread("image/3.jpg")
img_4 = cv2.imread("image/4.jpg")
img_5 = cv2.imread("image/5.jpg")
img_6 = cv2.imread("image/6.jpg")
img_7 = cv2.imread("image/7.jpg")
img_8 = cv2.imread("image/8.jpg")
img_9 = cv2.imread("image/9.jpg")

In [4]:
def plot_img(img1, img2, img3, img4 = None):
    fig = plt.figure(num=None, figsize=(8, 8), dpi=100, facecolor='w', edgecolor='k')

    fig.add_subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))

    fig.add_subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    plt.show()

    plt.figure(num=None, figsize=(6, 6), dpi=100, facecolor='w', edgecolor='k')
    plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
    plt.show()

    if img4 is not None:
        plt.figure(num=None, figsize=(6, 6), dpi=100, facecolor='w', edgecolor='k')
        plt.imshow(cv2.cvtColor(img4, cv2.COLOR_BGR2RGB))
    plt.show()

In [5]:
label_path = "models/voc-model-labels.txt"
model_path = "models/mobilenet-v1-ssd-mp-0_675.pth"

# label_path = "models/voc-model-labels.txt"
# model_path = "models/mb2-ssd-lite-mp-0_686.pth"

In [6]:
class_names = [name.strip() for name in open(label_path).readlines()]
num_classes = len(class_names)

net = create_mobilenetv1_ssd(len(class_names), is_test=True)
predictor = create_mobilenetv1_ssd_predictor(net, candidate_size=200)

# net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)
# predictor = create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200)

net.load(model_path)

In [51]:
pts1 = []
pts2 = []
count = [0]
def click_Mouse(event, x, y, flags, param):
    global click_p
    if event == cv2.EVENT_LBUTTONUP:
        if count[0] < 4:
            pts1.append([x, y])
        else:
            pts2.append([x, y])
        count[0] += 1

cv2.namedWindow("vector", 0)
cv2.resizeWindow("vector", 750, 750);
cv2.setMouseCallback("vector", click_Mouse)
cv2.imshow("vector", frame)
# ch = cv2.waitKey(1)
# if ch & 0xFF == ord('q'):
cv2.waitKey(0)
cv2.destroyAllWindows() 
pts1 = np.float32(pts1) 
pts2 = np.float32(pts2)

In [146]:
# Create capture object
cap = cv2.VideoCapture(5) # Flush the stream

#cap = cv2.VideoCapture(1) # Then start the webcam
# cap = cv2.VideoCapture('video/wind_rain.mp4')
cap = cv2.VideoCapture('video/normal_night.mp4')
# cap = cv2.VideoCapture('video/dark.mp4')
# cap = cv2.VideoCapture('video/622.mp4')
# cap = cv2.VideoCapture(0)   # capture from camera
# cap.set(3, 1920)
# cap.set(4, 1080)

cv2.namedWindow("frame",0);
cv2.resizeWindow("frame", 750, 750);
M = cv2.getPerspectiveTransform(pts1,pts2)
M_shift = np.eye(3)
M_shift[0,2] = 1327 # x
M_shift[1,2] = 385 # y
M_ = np.matmul(M_shift, M)
while(cap.isOpened()):
    ret, frame = cap.read()
    if (frame is not None):
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes, labels, probs = predictor.predict(image, 10, 0.25)
        for i in range(boxes.size(0)):
            box = boxes[i, :]
            label = f"{class_names[labels[i]]}: {probs[i]:.2f}"
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 255, 0), 4)

            cv2.putText(frame, label,
                        (box[0]+20, box[1]+40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,  # font scale
                        (255, 0, 255),
                        2)  # line type

        frame = cv2.warpPerspective(frame, M_,(1800+1327, 1080+385))
#         frame = cv2.warpPerspective(frame, M_,(frame.shape[1], frame.shape[0]))
        cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [122]:
min_x = 1000
min_y = 1000
max_x = -1000
max_y = -1000
for i in range(frame.shape[0]): # y
    for j in range(frame.shape[1]): # x
        p = np.matmul(M, [j, i, 1])
        min_x = min(min_x, p[0])
        min_y = min(min_y, p[1])
        max_x = max(max_x, p[0])
        max_y = max(max_y, p[1])

In [144]:
np.matmul(M, [1919, 0, 1])

array([ 1.62788359e+03, -3.85010974e+02,  1.04491859e+00])

In [123]:
print(min_x, min_y, max_x, max_y)
print(frame.shape)

-1327.393590443272 -385.62224901508836 3764.2140761975525 2432.606865428087
(1080, 1920, 3)
